In [1]:
vertex_data = None
with open('VertexShader_scale.fxc', 'rb') as f:
    vertex_data = f.read()

In [7]:
print(hex(int.from_bytes(vertex_data[:4], 'little')))

0xfffe0300


In [9]:
version_token = int.from_bytes(vertex_data[:4], 'little')
minor = version_token&0xff
major = (version_token>>8)&0xff
# pixel shader: 0xFFFF
# vertex shader: 0xFFFE
shadertype = (version_token>>16)&0xffff
print('%s %02d%02d' % ('vs' if shadertype==0xfffe else 'ps', major, minor))

vs 0300


In [12]:
comment_token = int.from_bytes(vertex_data[4:8], 'little')
print(hex(comment_token))
# [15:00] is 0xFFFE
# [30:16] length
# [31] must be zero
tokcount = (comment_token>>16)&0xffff
print('count:%d' % tokcount)

0x21fffe
count:33


In [14]:
print(vertex_data[8:12])
if vertex_data[8:12].decode('utf-8') == 'CTAB':
    print('Have constant_table')

b'CTAB'
Have constant_table


In [16]:
size = int.from_bytes(vertex_data[12:16], 'little')
creator = int.from_bytes(vertex_data[16:20], 'little')
version = int.from_bytes(vertex_data[20:24], 'little')
constants = int.from_bytes(vertex_data[24:28], 'little')
constantinfo = int.from_bytes(vertex_data[28:32], 'little')
flag = int.from_bytes(vertex_data[32:36], 'little')
target = int.from_bytes(vertex_data[36:40], 'little')
print('size:%d creator:%d version:0x%x constants:%d constantinfo:%d target:%d' % (size, creator, version, 
                    constants, constantinfo, target))

size:28 creator:79 version:0xfffe0300 constants:1 constantinfo:28 target:72


In [33]:
# constant
name = int.from_bytes(vertex_data[40:44], 'little')
regset = int.from_bytes(vertex_data[44:46], 'little')
regidx = int.from_bytes(vertex_data[46:48], 'little')
regcnt = int.from_bytes(vertex_data[48:50], 'little')
reserved = int.from_bytes(vertex_data[50:52], 'little')
typeinf = int.from_bytes(vertex_data[52:56], 'little')
defval = int.from_bytes(vertex_data[56:60], 'little')
print('name:%d regset:%d regidx:%d regcnt:%d typeinf:%d defval:%d' % (name, regset, regidx, regcnt,
                                        typeinf, defval))

name:48 regset:2 regidx:0 regcnt:1 typeinf:56 defval:0


In [34]:
# 获取常量名称start+name = 12 + 48
name_end = vertex_data.find(0, 60)
constant_name = vertex_data[60:name_end].decode('utf-8')
print(constant_name)

scale


In [38]:
# ctab_typeinfo 位置:start+typeinf = 12 + 56 = 68
parameter_class = int.from_bytes(vertex_data[68:70], 'little')
parameter_type = int.from_bytes(vertex_data[70:72], 'little')
rows = int.from_bytes(vertex_data[72:74], 'little')
columns = int.from_bytes(vertex_data[74:76], 'little')
elements = int.from_bytes(vertex_data[76:78], 'little')
member_count = int.from_bytes(vertex_data[78:80], 'little')
memberinfo = int.from_bytes(vertex_data[80:84], 'little')
print('class:%d type:%d rows:%d cloumns:%d elements:%d member_count:%d memberinfo:%d' % (parameter_class,
                            parameter_type, rows, columns, elements, member_count, memberinfo))

class:0 type:3 rows:1 cloumns:1 elements:1 member_count:0 memberinfo:0


In [47]:
# member_count为0，不用再解析StructMember
# tokcount为33，则结尾为8+33*4=140
print(vertex_data[84:140].decode('utf-8'))

vs_3_0 Microsoft (R) HLSL Shader Compiler 9.29.952.3111 
